In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from slugify import slugify 

In [80]:
from ast import literal_eval

In [2]:
import subprocess
import pyalex
def get_git_user():
    res = subprocess.run(["git", "config", "user.email"], stdout=subprocess.PIPE)
    return res.stdout.strip().decode()

## 1. Load list of relevant journals

In [3]:
journals_list = pd.read_csv("../reviewerSelection-data/JCR_JournalResults_Sociology.csv")
journals_list.head()

,Journal name,JCR Abbreviation,ISSN,eISSN,Category,Total Citations,2022 JIF,JIF Quartile,2022 JCI,% of OA Gold
0,ANNALS OF TOURISM RESEARCH,ANN TOURISM RES,0160-7383,1873-7722,SOCIOLOGY - SSCI,"19,874",13.2,Q1,3.52,17.21%
1,Annual Review of Sociology,ANNU REV SOCIOL,0360-0572,1545-2115,SOCIOLOGY - SSCI,"15,214",10.5,Q1,4.30,1.15%
2,AMERICAN SOCIOLOGICAL REVIEW,AM SOCIOL REV,0003-1224,1939-8271,SOCIOLOGY - SSCI,"25,619",9.1,Q1,4.98,13.91%
3,EUROPEAN SOCIETIES,EUR SOC,1461-6696,1469-8307,SOCIOLOGY - SSCI,"2,158",8.1,Q1,2.54,34.97%
4,SOCIOLOGICAL METHODS & RESEARCH,SOCIOL METHOD RES,0049-1241,1552-8294,SOCIOLOGY - SSCI,"8,840",6.3,Q1,3.18,15.87%


In [4]:
'''try_journals = journals_list.loc[[1, 3]]
try_journals_issn = try_journals.filter(items=['ISSN'])
try_journals_issn'''

"try_journals = journals_list.loc[[1, 3]]\ntry_journals_issn = try_journals.filter(items=['ISSN'])\ntry_journals_issn"

In [5]:
'''total_list = []
for issn in try_journals_issn['ISSN']:    
    curr_res_list = []
    cursor = '*'
    select = 'id,authorships,publication_year,display_name,primary_location,cited_by_count,language'
    while cursor != None:
        filter = f'primary_location.source.issn:{issn}'
        params = {
            'filter': filter,
            'per_page': 200,
            'cursor': cursor,
            'select': select
        }
        res = requests.get("https://api.openalex.org/works", params=params).json()
        for work in range(len(res['results'])): 
            curr_res = res['results'][work]
            curr_res_list.append(curr_res)
        cursor = res['meta'].get('next_cursor', None)
        print(res)
        print(cursor)
        print(len(curr_res_list))
    
    total_list.append(curr_res_list)
    print((len(total_list)))'''

'total_list = []\nfor issn in try_journals_issn[\'ISSN\']:    \n    curr_res_list = []\n    cursor = \'*\'\n    select = \'id,authorships,publication_year,display_name,primary_location,cited_by_count,language\'\n    while cursor != None:\n        filter = f\'primary_location.source.issn:{issn}\'\n        params = {\n            \'filter\': filter,\n            \'per_page\': 200,\n            \'cursor\': cursor,\n            \'select\': select\n        }\n        res = requests.get("https://api.openalex.org/works", params=params).json()\n        for work in range(len(res[\'results\'])): \n            curr_res = res[\'results\'][work]\n            curr_res_list.append(curr_res)\n        cursor = res[\'meta\'].get(\'next_cursor\', None)\n        print(res)\n        print(cursor)\n        print(len(curr_res_list))\n    \n    total_list.append(curr_res_list)\n    print((len(total_list)))'

In [6]:
'''end_list = []
for sec_list in total_list:
    for third_list in sec_list:
        end_list.append(third_list)'''

'end_list = []\nfor sec_list in total_list:\n    for third_list in sec_list:\n        end_list.append(third_list)'

In [7]:
'''all_ids = []
all_authors = []
all_titles = []
all_pub_years = []
all_journal_names = []
all_journal_issnls = []
all_cited_counts = []
all_languages = []


for work in end_list:
    # IDs
    id = work['id']
    # Title
    title = work['display_name']
    # Author list
    authors = []
    for author in work['authorships']:
        authors.append(author['author']['display_name'])
    # Publication year
    pub_year = work['publication_year']
    # Journal Name
    journal_name = work['primary_location']['source']['display_name']
    # Journal Issn_l
    journal_issnl = work['primary_location']['source']['issn_l']
    # Cited by count
    cited_count = work['cited_by_count']
    # Language
    language = work['language']
    
    # Append to each list
    all_ids.append(id)
    all_titles.append(title)
    all_authors.append(authors)
    all_pub_years.append(pub_year)
    all_journal_names.append(journal_name)
    all_journal_issnls.append(journal_issnl)
    all_cited_counts.append(cited_count)
    all_languages.append(language)'''
    

"all_ids = []\nall_authors = []\nall_titles = []\nall_pub_years = []\nall_journal_names = []\nall_journal_issnls = []\nall_cited_counts = []\nall_languages = []\n\n\nfor work in end_list:\n    # IDs\n    id = work['id']\n    # Title\n    title = work['display_name']\n    # Author list\n    authors = []\n    for author in work['authorships']:\n        authors.append(author['author']['display_name'])\n    # Publication year\n    pub_year = work['publication_year']\n    # Journal Name\n    journal_name = work['primary_location']['source']['display_name']\n    # Journal Issn_l\n    journal_issnl = work['primary_location']['source']['issn_l']\n    # Cited by count\n    cited_count = work['cited_by_count']\n    # Language\n    language = work['language']\n    \n    # Append to each list\n    all_ids.append(id)\n    all_titles.append(title)\n    all_authors.append(authors)\n    all_pub_years.append(pub_year)\n    all_journal_names.append(journal_name)\n    all_journal_issnls.append(journal_is

In [8]:
'''df = pd.DataFrame({
    'id': all_ids,
    'authors': all_authors,
    'title': all_titles,
    'publication_year': all_pub_years,
    'journal_name': all_journal_names,
    'journal_issnl': all_journal_issnls,
    'cited_by_count': all_cited_counts,
    'language': all_languages
})'''

"df = pd.DataFrame({\n    'id': all_ids,\n    'authors': all_authors,\n    'title': all_titles,\n    'publication_year': all_pub_years,\n    'journal_name': all_journal_names,\n    'journal_issnl': all_journal_issnls,\n    'cited_by_count': all_cited_counts,\n    'language': all_languages\n})"

In [9]:
'''df['authors_first_lastname'] = df['authors'].apply(lambda x: x[0].lower().replace('.', '').split()[-1] if len(x)>0 else '')
df['authors_first_lastname']'''

"df['authors_first_lastname'] = df['authors'].apply(lambda x: x[0].lower().replace('.', '').split()[-1] if len(x)>0 else '')\ndf['authors_first_lastname']"

In [10]:
'''df['title_slugified'] = df['title'].apply(lambda x: slugify(x) if type(x)==str else '')
df['title_slugified']'''

"df['title_slugified'] = df['title'].apply(lambda x: slugify(x) if type(x)==str else '')\ndf['title_slugified']"

In [11]:
'''df['concat_name_title'] = df['authors_first_lastname'] + '-' + df['title_slugified']
df['concat_name_title']'''

"df['concat_name_title'] = df['authors_first_lastname'] + '-' + df['title_slugified']\ndf['concat_name_title']"

In [12]:
'''df.to_csv('two_journals.csv')'''

"df.to_csv('two_journals.csv')"

## Extract works from OpenAlex

In [13]:
def extract_works_relevant(list_relevant):
    '''Returns dataframe of all works from OpenAlex from all journals in list_relevant.'''
    user_mail = get_git_user()
    current_date = '2023-08-29' #year-month-day
    year_limit = 2023 - 30  #year - 30
    date_limit = f'{year_limit}' + current_date[-6:]

    total_list = []
    for issn in list_relevant['ISSN']:
        
        curr_res_list = []
        cursor = '*'
        select = 'id,authorships,publication_year,display_name,primary_location,cited_by_count,language'
        
        while cursor != None:
            filters = f'primary_location.source.issn:{issn},from_publication_date:{date_limit},to_publication_date:{current_date}'
            params = {
                'filter': filters,
                'per_page': 200,
                'cursor': cursor,
                'select': select,
                'mailto': user_mail
            }
            res = requests.get("https://api.openalex.org/works", params=params).json()
            for work in range(len(res['results'])): 
                curr_res = res['results'][work]
                curr_res_list.append(curr_res)
            cursor = res['meta'].get('next_cursor', None)
            print(len(curr_res_list))

        total_list.append(curr_res_list)
        print(len(total_list))
    
    return total_list

In [14]:
def create_df(total_list):
    '''Create a dataframe out of the works in pages in a list. Returns df.''' 
    
    #Create list of works
    end_list = []
    for sec_list in total_list:
        for third_list in sec_list:
            end_list.append(third_list)
    
    all_ids = []
    all_titles = []
    all_authors = []
    all_pub_years = []
    all_journal_names = []
    all_journal_issnls = []
    all_cited_counts = []
    all_languages = []


    for work in end_list:
        # IDs
        id = work['id']
        # Title
        title = work['display_name']
        # Author list
        authors = []
        for author in work['authorships']:
            authors.append(author['author'].get('display_name', ''))
        # Publication year
        pub_year = work['publication_year']
        # Journal Name
        journal_name = work['primary_location']['source']['display_name']
        # Journal Issn_l
        journal_issnl = work['primary_location']['source']['issn_l']
        # Cited by count
        cited_count = work['cited_by_count']
        # Language
        language = work['language']
        
        # Append to each list
        all_ids.append(id)
        all_titles.append(title)
        all_authors.append(authors)
        all_pub_years.append(pub_year)
        all_journal_names.append(journal_name)
        all_journal_issnls.append(journal_issnl)
        all_cited_counts.append(cited_count)
        all_languages.append(language)
        
    #Turn into df
    df = pd.DataFrame({
    'id': all_ids,
    'authors': all_authors,
    'title': all_titles,
    'publication_year': all_pub_years,
    'journal_name': all_journal_names,
    'journal_issnl': all_journal_issnls,
    'cited_by_count': all_cited_counts,
    'language': all_languages
    })
       
    return df

In [88]:
def concat_author_title(df):
    '''Add a column with the first authors last name and title of the work to the dataframe. Returns dataframe.'''
    df['authors_first_lastname'] = df['authors'].apply(lambda x: x[0].lower().replace('.', '').split()[-1] if len(x)>0 and x[0] is not None else '')

    df['title_slugified'] = df['title'].apply(lambda x: slugify(x) if type(x)==str else '')
    df['concat_name_title'] = df['authors_first_lastname'] + '-' + df['title_slugified']
    
    return df

In [16]:
all_journals = journals_list.filter(items=['ISSN'])



In [17]:
df_step1 = extract_works_relevant(all_journals)


200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3665
3665
1
200
400
600
792
792
2
200
400
600
800
1000
1200
1400
1600
1764
1764
3
200
400
600
800
972
972
4
200
400
600
800
978
978
5
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3364
3364
6
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2474
2474
7
200
218
218
8
200
400
600
773
773
9
200
400
600
800
1000
1049
1049
10
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5508
5508
11
200
400
600
800
1000
1200
1400
1600
1800
2000
2113
2113
12
200
400
600
693
693
13
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6475
6475
14
200
400
600
686
686
15
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2590
2590
16
200
400
600
800
1000
1098
1098
17
200
233
233
18
200
400
600
800
10

AttributeError: 'list' object has no attribute 'to_csv'

In [19]:

df_step2 = create_df(df_step1)


In [20]:
df_step2.to_csv('work_data/all_journals.csv')

In [76]:
df_step3 = pd.read_csv('work_data/all_journals.csv')

In [83]:

df_step3['authors'] = df_step3['authors'].apply(literal_eval)

In [89]:

end_df = concat_author_title(df_step3)

In [91]:
end_df.to_csv('work_data/all_works_sociology.csv')